В данной работе описана модель GenBERT для решения NLP-задач, которые требуют арифметических вычислений.

В последнее время получили распространение языковые модели - нейронные сети, обученные на задачах языкового моделирования. Например, модель GPT обучается на задаче, в которой нужно предсказывать продолжение текста, а модель BERT обучается сразу на нескольких задачах: определять слова, закрытые маской, исправлять слова, замененные случайным образом, а также определять, является ли один текст прямым продолжением другого. В результате такие модели научаются хорошо работать с текстами и могут быть дообучены для решения различных практических задач. Более подробно я писал о GPT и BERT в [одном из предыдущих обзоров]($GPT и BERT$).

Однако модели, обученные на задаче языкового моделирования, испытывают трудности при решении арифметических задач (numerical reasoning over text, NRoT). Например, датасет [DROP](https://paperswithcode.com/dataset/drop) состоит из текстов, вопросов и ответов по ним. К примеру, тексту по теме американского футбола соответствуют такие вопросы:

- "How many total yards did Phil Dawson throw for touchdowns?"
- "Who threw 45 total yards for touchdowns?"

В первом случае ответом является результат вычислений, а во втором случае ответом является имя, для его получения которого нужно выполнить вычисления. Модель BERT сама по себе дает не очень хорошие результаты при файн-тюнинге на датасете DROP. В данной работе авторы решают эту проблему с помощью нового способа токенизации чисел и предобучения на синтетическом датасете, состоящем из арифметических задач.

### Токенизация цифр (digit tokenization)

Способ токенизации [WordPiece]($Googles Neural Machine Translation System: Bridging the Gap between Human and Machine Translation$), используемый в BERT, работает с числами так же, как со словами. Авторы текущей работы предлагают другой способ, в котором токенами являются отдельные цифры в числе. Это означает, что для каждой цифры вводится эмбеддинг, и число 123 будет преобразовано в последовательность из трех векторов: эмбеддинг числа 1, эмбеддинг числа 2 и эмбеддинг числа 3.

Конечно можно придумать и другие способы создания эмбеддингов чисел. Например, можно превращать число в один вектор-эмбеддинг, складывая эмбеддинг нуля и эмбеддинг единицы, умноженный на данное число. Но предложенный авторами способ хорош во-первых тем, что даже при работе с очень большими числами нормы эмбеддингов никогда не будут стремиться к бесконечности, а во-вторых тем, что иногда числа могут быть представлены в виде слов ("fifty nine and a half"), и способ кодирования чисел, записанных словами или цифрами, получается согласованным.

### Синтетические датасеты для предобучения

Авторами разработаны два синтетических (автоматически генерируемых) датасета, которые используются для предобучения модели GenBERT. Датасет **Numerical Data (ND)** состоит только из операций над числами и датами.

<img src="assets/genbert.jpg" width="800" align="center">

При обучении на датасете Numerical Data осуществлялся случайный сдвиг входных данных на величину до 512 токенов. Или, иначе говоря, позиционные эмбеддинги (см. [Attention Is All You Need]($Attention Is All You Need$), раздел "Позиционное кодирование") сдвигались в обратную сторону. Это делалось для того, чтобы модель умела работать с числами, находящимися на любой позиции в тексте.

Еще один датасет **Textual Data (TD)** содержит контексты и вопросы, для получения ответов на которые требуется выполнять арифметические операции.

<img src="assets/genbert2.jpg" width="400" align="center">

Со своей стороны отмечу возможный недостаток такого подхода: модель при предобучении накапливает информацию, взятую из текстов, и предобучение на таком датасете может повлиять на то, верные ли исторические факты запомнит модель. С другой стороны, высокая вариативность чисел и слов в синтетическом датасете может сгладить эту проблему.

### Архитектура и обучение модели GenBERT

Синтетические датасеты построены либо как пары (вопрос + ответ), либо как тройки (контекст + вопрос + ответ). В некоторых случаях ответ может быть найден в вопросе, то есть достаточно указать промежуток (span) в вопросе, который и является ответом. В этом случае для каждого токена модель выдает вероятности того, что он является началом или концом span'а. Более подробно о формате входных данных, формате выходных данных и функции потерь в такой постановке задачи я писал в [обзоре на BERT]($GPT и BERT$), раздел "Дообучение на SQuAD". В данном случае ситуация аналогичная.

В некоторых случаях ответ может не содержаться в вопросе, например если нужно посчитать сумму чисел. В этом случае ответ нужно сгенерировать, и для этого модель дополняется декодером трансформера (подробнее об энкодере и декодере я писал в [этом обзоре]($Attention Is All You Need$); для понимания принципа работы энкодера и декодера достаточно прочитать раздел "Общее устройство трансформера"). Декодер в GenBERT использует те же веса, что и энкодер. Self-attention блок в декодере, направленный на энкодер, использует те же веса, что и другой self-attention блок, направленный на сам декодер. Чтобы энкодер и декодер могли работать по-разному, на выходе энкодера и декодера добавляется еще один полносвязный слой с функцией активации GeLU и операцией LayerNorm. Этот слой инициалирзируется разными весами в энкодере и декодере (на схеме слой обозначен как **FFenc** и **FFdec**).

<img src="assets/genbert3.jpg" width="400" align="center">

Модель GenBERT при обучении на синтетическом датасете использует четыре головы:

1. Голова **question span** присоединена к выходу энкодера и ищет ответ в вопросе, то есть выдает распределение вероятностей начала и конца span'а по выходным токенам, соответствующим вопросу (подробнее см. в [обзоре на BERT]($GPT и BERT$), раздел "Дообучение на SQuAD"). На схеме вопросу соответствуют входные токены $q_i$.
2. Голова **context span** присоединена к выходу энкодера и таким же образом ищет ответ в контексте, то есть в тексте, по которому задан вопрос.  На схеме контексту соответствуют входные токены $p_i$.
3. **Выход декодера** является третьей головой.
4. Голова **type** присоединена к выходу энкодера и выдает распределение вероятностей по остальным трем головам, то есть отвечает на вопрос о том, с какой головы нужно считывать ответ.

Функция потерь в декодере является стандартной (см. [обзор на трансформер]($Attention Is All You Need$), раздел "Принцип обучения трансформера"). Формально условная вероятность отета $\langle a \rangle$, то есть последовательности токенов $[SOS], a_1, \dots, a_m, [EOS]$, при контексте $\textbf{c}$ и вопросе $\textbf{q}$ вычисляется с помощью декодера следующим образом:

$p_\text{dec}(\langle a \rangle\ |\ \textbf{c}, \textbf{q}) = \prod\limits_{i=0}^m p_\text{dec}(a_{i+1} | a_0, \dots, a_i, \textbf{c}, \textbf{q})$

Каждая голова выдает распределение вероятностей по возможным ответам, и умножая это распределение на вероятность данной головы (выданную головой **type**), мы получаем финальные вероятности. В качестве функции потерь используется кроссэнтропия (logloss), то есть минимизация минус логарифма вероятности верного ответа.

Формально общая функция потерь вычисляется таким образом:

$-\log \Big( \textbf{p}_\textbf{q} p_\text{dec}(\langle a \rangle\ |\ \textbf{c}, \textbf{q}) + \textbf{p}_\textbf{q} \sum\limits_{(i, j) \in S} p_q (i, j\ |\ \textbf{c}, \textbf{q}) + \textbf{p}_\textbf{c} \sum\limits_{(i, j) \in S} p_c (i, j\ |\ \textbf{c}, \textbf{q}) \Big)$

Фактически в данной формуле вероятность считается как взвешенное среднее по трем головам, где веса - вероятности голов. Вектор [$\textbf{p}_\textbf{q}$, $\textbf{p}_\textbf{c}$, $\textbf{p}_\textbf{dec}$] - это эталонное распределение вероятностей по трем головам, взятым из разметки (вероятно это единица для верной головы и ноль для неверных голов).

Важно также, что в процессе обучения на синтетическом датасете модель GenBERT параллельно *продолжает обучаться на задаче языкового моделирования* (на которой обучалась BERT), чтобы модель не "забыла", как работать с текстом, не содержащим числа. На каждом шаге обучения батч обучающих данных является конкатенацией трех батчей:

1. Батч из синтетического датасета Numerical Data (ND)
2. Батч из синтетического датасета Textual Data (TD)
3. Батч из задачи Masked Language Model (MLM) (подробнее см. [GPT и BERT]($GPT и BERT$)).

Суммарная функция потерь при этом является взвешенной суммой трех функций потерь по этим трем типам задач.

**Достигнутые результаты**

Авторы сравнивают две модели:
1. Оригинальную модель BERT
2. Модель BERT, которая дообучалась на синтетическом датасете с использованием токенизацией цифр

Вторая модель показывала намного более высокое качество при файн-тюнинге на датасете DROP (о котором упоминалось выше). Это происходит без потери качества файн-тюнинга на других задачах. Чтобы проверить это, авторы файн-тюнили обе модели на датасете SQuAD, где не требуется работа с числами, и получали примерно одинаковые результаты.

Преимущество модели GenBERT в том, что она достаточно проста и универсальна, при этом умея явно или неявно работать с числами и датами: складывать, вычитать, искать средние, максимальные и минимальные значения. Тем не менее, модель не обучена для решения некоторых других классов задач, таких как сортировка чисел, а также плохо работает с арифметическими выражениями, где встречается более трех чисел (см. разделы 5.1, 5.2 статьи).

Авторы предполагают, что эти ограничения могут быть преодолены путем добавления в синтетический датасет новых типов примеров. Хотя, с моей точки зрения, остается неясным: не будет ли необходимый объем предобучения расти экспоненциально в зависимости от длины арифметических выражений? Если будет, то для работы с длинными арифметическими выражениями нужен иной подход.